I set up the data to give continuous normal loads. Instead of classifying by the discrete normal loads (the ones specified), I've included all values for a continuous spectrum.

In [20]:
import scipy.io as sio
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D

In [21]:
tires = {"hoosier_r25b_18x6_10x7":{"long":None, "lat":None}}

camber = 0 # default camber
pressure = 12 * 6.89476 # default pressure
velocity = 25 * 1.60934 # default velocity
slip_angle = 0
run_num = 1


In [22]:
for name, tire in tires.items():
    try:
        df = pd.read_csv(f"../tire_data/processed_data/braking_{name}.csv")
        tire["long"] = df[(df["pressure"] == pressure) & (df["velocity"] == velocity) & (df["camber"] == camber) & (df["slip"] == slip_angle)]
        
    except:
        print("Error getting long data for {0}".format(name))

    try:
        df = pd.read_csv(f"../tire_data/processed_data/cornering_{name}.csv")
        tire["lat"] = df[((df["velocity"] == velocity) & (df["pressure"] == pressure) & (df["camber"] == camber))]

    except:
        print("Error getting lateral data for {0}".format(name))

In [36]:
df = tires["hoosier_r25b_18x6_10x7"]["lat"]

raw_points = [df["FZ"].unique().tolist(), df["SA"].tolist(), df["MZ"].tolist()]

temp_df = df[(df["load"] == -93 / 0.224809)]

def function(data, a, b, c):
    x = data[0]
    y = data[1]
    return a * (x**b) * (y**c)

parameters, covariance = curve_fit(function, [raw_points[0] + raw_points[1]], raw_points[2])


model_x_data = np.linspace(min(raw_points[0]), max(raw_points[0]), 30)
model_y_data = np.linspace(min(raw_points[1]), max(raw_points[1]), 30)
# create coordinate arrays for vectorized evaluations
X, Y = np.meshgrid(model_x_data, model_y_data)
# calculate Z coordinate array
Z = function(np.array([X, Y]), *parameters)

# setup figure object
fig = plt.figure()
# setup 3d object
ax = Axes3D(fig)
# plot surface
ax.plot_surface(X, Y, Z)
# plot input data
ax.scatter([raw_points[0]], raw_points[1], raw_points[2], color='red')
# set plot descriptions
ax.set_xlabel('X data')
ax.set_ylabel('Y data')
ax.set_zlabel('Z data')

plt.show()




curve = np.poly1d(np.polyfit(temp_df["SA"], temp_df["MZ"], 10))

polyline = np.linspace(temp_df["SA"].min(), temp_df["SA"].max(), 50)

plt.plot(polyline, curve(polyline))
plt.scatter(temp_df["SA"], temp_df["MZ"], s=1)
# plt.plot(temp_df["SA"], temp_df["MZ"])
plt.ylabel("Aligning Moment")
plt.xlabel("Slip Ratio (%)")

IndexError: index 1 is out of bounds for axis 0 with size 1